In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/content/train.csv",engine='python',error_bad_lines=False)

<ipython-input-4-9442758d9a1e>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv("/content/train.csv",engine='python',error_bad_lines=False)
Skipping line 1259: unexpected end of data


In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum()

id          0
title      36
author    132
text        3
label       0
dtype: int64

In [ ]:
df.shape

(1257, 5)

In [ ]:
df=df.dropna()

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
X=df.drop('label',axis=1)

In [ ]:
y=df['label']

In [ ]:
y.value_counts()

0    619
1    470
Name: label, dtype: int64

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [ ]:
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
X.head(1)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
1084,1249,1249,Universal Child Season 1 Episode 24 Hosted by ...,Madeline,"Posted by Madeline | Oct 30, 2016 | 2016 , Dai..."
1085,1251,1251,"Price for a Green Card: $500,000 Stadium Stake...",Ken Belson,"ORLANDO, Fla. — For years, sports teams hav..."
1086,1252,1252,Michael Moore: Forget What I Said Before. Trum...,EdJenner,Go to Article \nFilmmaker Michael Moore made a...
1087,1255,1255,The Lethal Lie of Hillary Clinton: Saving Live...,shorty,ABOVE: Image by DonkeyHotey | CC BY 2.0 “We mu...


In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
  print(i)
  review=re.sub('[^a-zA-Z]',' ',messages['text'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [ ]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel case know anthoni weiner sext teenag appar littl thing fact matter ch

In [ ]:
df=pd.DataFrame(corpus)

In [ ]:
df.to_csv("/content/corpus.csv")

In [ ]:
m=0
for i in corpus:
  m=max(m,len(i))
print(m)

51288


In [ ]:
sent_length=51300
embeddded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [ ]:
embedding_vector_features=20
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 51300, 20)         100000    
                                                                 
 bidirectional (Bidirection  (None, 200)               96800     
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 197001 (769.54 KB)
Trainable params: 197001 (769.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
import numpy as np
x_final=np.array(embeddded_docs)
y_final=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=42)


In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
23/23 [==============================] - 87s 4s/step - loss: 0.6824 - accuracy: 0.5679 - val_loss: 0.6725 - val_accuracy: 0.5528
Epoch 2/10
23/23 [==============================] - 80s 3s/step - loss: 0.6391 - accuracy: 0.6351 - val_loss: 0.6262 - val_accuracy: 0.6222
Epoch 3/10
23/23 [==============================] - 87s 4s/step - loss: 0.3921 - accuracy: 0.8573 - val_loss: 0.4354 - val_accuracy: 0.7667
Epoch 4/10
23/23 [==============================] - 88s 4s/step - loss: 0.1339 - accuracy: 0.9712 - val_loss: 0.4771 - val_accuracy: 0.8056
Epoch 5/10
23/23 [==============================] - 87s 4s/step - loss: 0.0573 - accuracy: 0.9890 - val_loss: 0.3939 - val_accuracy: 0.8472
Epoch 6/10
23/23 [==============================] - 79s 3s/step - loss: 0.0500 - accuracy: 0.9877 - val_loss: 0.4126 - val_accuracy: 0.8500
Epoch 7/10
23/23 [==============================] - 87s 4s/step - loss: 0.0133 - accuracy: 0.9986 - val_loss: 0.3938 - val_accuracy: 0.8556
Epoch 8/10
23/23 [==